In [1]:
import tensorflow as tf
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import os

data_path = '/Users/jiayunwang/Documents/datasets/10_food_classes_10_percent/'
# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk(data_path):
    class_name = '/'.join(dirpath.split("/")[-2:])
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{class_name}'.")

There are 2 directories and 1 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in 'test/ice_cream'.
There are 0 directories and 250 images in 'test/chicken_curry'.
There are 0 directories and 250 images in 'test/steak'.
There are 0 directories and 250 images in 'test/sushi'.
There are 0 directories and 250 images in 'test/chicken_wings'.
There are 0 directories and 250 images in 'test/grilled_salmon'.
There are 0 directories and 250 images in 'test/hamburger'.
There are 0 directories and 250 images in 'test/pizza'.
There are 0 directories and 250 images in 'test/ramen'.
There are 0 directories and 250 images in 'test/fried_rice'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in 'train/ice_cream'.
There are 0 directories and 75 images in 'train/chicken_curry'.
There are 0 directories and 75 images in 'train/steak'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dir = data_path + 'train/'
test_dir = data_path + 'test/'

train_data_gen = ImageDataGenerator(rescale=1/255.)
test_data_gen = ImageDataGenerator(rescale=1/255.)

print('Training images:')
train_data_10_percent = train_data_gen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SIZE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')
print('Test images:')
test_data_10_percent = test_data_gen.flow_from_directory(test_dir,
                                                        target_size=IMAGE_SIZE,
                                                        batch_size=BATCH_SIZE,
                                                        class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.
Test images:
Found 2500 images belonging to 10 classes.


In [4]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + '/' + experiment_name + '/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(lot_dir=log_dir)
    print(f'Saving TensorBoard log files to: {log_dir}')
    return tensorboard_callback

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [6]:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [7]:
def create_model(model_url, num_classes=10):
    """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.

    Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

    Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
    """
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SIZE+(3,)) # define the input image shape

    # Create our own model
    model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer      
    ])

    return model

In [10]:
# Create model
# download is slow
resnet_model = create_model(resnet_url, num_classes=train_data_10_percent.num_classes)

# # Compile
# resnet_model.compile(loss='categorical_crossentropy',
#                      optimizer=tf.keras.optimizers.Adam(),
#                      metrics=['accuracy'])